In [91]:
from datasets import load_from_disk

In [20]:
cbn_wiki_id = load_from_disk("dataset/paralel_id_cbn_16k")
cbn_wiki_jv = load_from_disk("dataset/paralel_jv_cbn_3k")
cbn_wiki_su = load_from_disk("dataset/paralel_su_cbn_3k")

In [21]:
id_wiki_id = load_from_disk("synthetic/id_titles/id_wiki-id/translated")
id_wiki_jv = load_from_disk("synthetic/id_titles/id_wiki-jv/translated")
id_wiki_su = load_from_disk("synthetic/id_titles/id_wiki-su/translated")

In [22]:
from datasets import concatenate_datasets

id_cbn = concatenate_datasets([cbn_wiki_id, id_wiki_id])
jv_cbn = concatenate_datasets([cbn_wiki_jv, id_wiki_jv])
su_cbn = concatenate_datasets([cbn_wiki_su, id_wiki_su])

In [25]:
len(su_cbn)

28551

In [26]:
id_cbn.save_to_disk("dataset/paralel_id_cbn_127k")
jv_cbn.save_to_disk("dataset/paralel_jv_cbn_24k")
su_cbn.save_to_disk("dataset/paralel_su_cbn_28k")

Saving the dataset (1/1 shards): 100%|██████████| 28551/28551 [00:00<00:00, 198725.47 examples/s]


In [27]:
paralel_bali_dict = load_from_disk("dataset/paralel_dataset_from_bali_dict")  

In [29]:
paralel_bali_dict = paralel_bali_dict.remove_columns(["prompt"])

In [31]:
paralel_60k = load_from_disk("dataset/paralel_dataset_60k")

In [33]:
def convert_dataset(data):
    data["id"] = data["custom_id"]
    return data

In [35]:
paralel_60k = paralel_60k.map(convert_dataset, remove_columns=["custom_id", "text", "prompt_text"])

Map: 100%|██████████| 59257/59257 [00:08<00:00, 7163.49 examples/s] 


In [39]:
paralel_300k = load_from_disk("dataset/paralel_dataset_300k")

In [40]:
paralel_300k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 296203
})

In [41]:
paralel_bali_dict

Dataset({
    features: ['id', 'balinese', 'indonesian', 'cirebonese'],
    num_rows: 22248
})

In [42]:
paralel_60k

Dataset({
    features: ['indonesian', 'cirebonese', 'balinese', 'id'],
    num_rows: 59257
})

In [100]:
348695 + 43818

392513

In [95]:
paralel_377k = load_from_disk("dataset/paralel_3_lang/paralel_dataset_377k_dedup")

In [96]:
paralel_44k = load_from_disk("dataset/paralel_3_lang/paralel_dataset_44k")

In [97]:
paralel_377k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 348695
})

In [99]:
paralel_44k

Dataset({
    features: ['id', 'indonesian', 'cirebonese', 'balinese'],
    num_rows: 43818
})

In [101]:
from datasets import concatenate_datasets

paralel_400k = concatenate_datasets([paralel_377k, paralel_44k])

In [102]:
paralel_400k.save_to_disk("dataset/paralel_3_lang/paralel_dataset_400k")

Saving the dataset (4/4 shards): 100%|██████████| 392513/392513 [00:27<00:00, 14353.79 examples/s]


In [116]:
paralel_400k[456]

{'id': '826b756d-1984-4a9b-9e44-603ef375821c',
 'cirebonese': 'Benteng Tiworo punika salah satunggal badan peninggalan sajarah saking Karaton Tiworo. Badan lan abha saking kubu punika taksih bisa tag saksikan hinga babak yasa. Benteng Tiworo kapanggihan ing Kelurahan Waumere, anatap Tiworo Kepulauan, kabupaten Muna Barat, Sulawesi Tenggara.\n\nBenteng Tiworo kadamel saking watu-watu alit lan ageng ingkang asusun rapi, lan ngadeg kuwat ing pusér Kota anatap Tiworo Kepulauan. Dhuwuré bervariasi, antawis telu ngantos papat meter. Kubu punika akělit amba kirang langkung loro hektar. Lokasi punika pinakangga saking imamat Karaton Tiworo. Hinga sapunika lokasi Benteng Tiworopun taksih disaklarkan, sanadyan badan aslinipun sampun ngalami owah-owahan.\n\nSajarah\nMiturut sajarah, Benteng Tiworo dibangun ing abad XVI dening Raja Muna ingkang La Ode Asmana. Pembuatan kubu punika saking bahan watu ingkang dikangkat masyarakat kalawan cara serendeng sepanjang 150 kilometer. Watu-watu ingkang dipun

In [52]:
id_cbn = load_from_disk("dataset/paralel_2_lang/paralel_id_cbn_127k_dedup")

In [53]:
existing_id_text = set(paralel_377k["indonesian"])

In [54]:
def filter_id_cbn(data):
    return data["text"] not in existing_id_text

In [55]:
filtered_id_cbn = id_cbn.filter(filter_id_cbn)

Filter: 100%|██████████| 92504/92504 [00:13<00:00, 6913.87 examples/s]


In [57]:
filtered_id_cbn.save_to_disk("dataset/paralel_2_lang/paralel_id_cbn_127k_filtered")

Saving the dataset (1/1 shards): 100%|██████████| 44743/44743 [00:00<00:00, 53675.53 examples/s]


In [78]:
from datasets import load_dataset

id_mmlu = load_dataset("indolem/IndoMMLU")

In [79]:
bali_mmlu = id_mmlu["test"].filter(lambda x: x["subject"] == "Balinese")

In [80]:
bali_mmlu[0]

{'subject': 'Balinese',
 'group': 'Local languages and cultures',
 'level': 'SD',
 'class': '2',
 'question': 'Dugas Galungane titiang mabakti nganggon udeng baru. Udeng basa Bali alusne...',
 'options': "['A. destar', 'B. kampuh', 'C. wastra']",
 'answer': 'A',
 'is_for_fewshot': '0'}

In [81]:
import ast
import uuid

def transform_data(item):
    question_id = str(uuid.uuid4())

    # Parse the options string into a list
    options_str = item['options']
    options_list = ast.literal_eval(options_str)
    
    # Extract labels and texts from options
    labels = []
    texts = []
    for opt in options_list:
        # Split by dot and strip whitespace
        parts = opt.split('.', 1)
        labels.append(parts[0].strip().lower())  # Get 'A', 'B', 'C' and convert to lowercase
        texts.append(parts[1].strip())  # Get the actual option text
    
    # Create the transformed dictionary
    transformed = {
        "context": "",
        "question": item['question'],
        "choices": {
            "label": labels,
            "text": texts
        },
        "answer": item['answer'].lower(),  # Convert answer to lowercase to match labels
        "category": [],
        "grade": int(item['class']),
        "question_id": question_id
    }
    
    return transformed

In [82]:
bali_mmlu = bali_mmlu.map(transform_data, remove_columns=['options', 'class', 'subject', 'group', 'level', 'class', 'is_for_fewshot'])

Map: 100%|██████████| 471/471 [00:00<00:00, 7187.59 examples/s]


In [83]:
bali_mmlu[0]

{'question': 'Dugas Galungane titiang mabakti nganggon udeng baru. Udeng basa Bali alusne...',
 'answer': 'a',
 'context': '',
 'choices': {'label': ['a', 'b', 'c'], 'text': ['destar', 'kampuh', 'wastra']},
 'category': [],
 'grade': 2,
 'question_id': 'e22caaf5-7596-4d76-a04e-61637f60a8c1'}

In [84]:
bali_mmlu.save_to_disk("dataset/bali_mmlu")

Saving the dataset (1/1 shards): 100%|██████████| 471/471 [00:00<00:00, 60177.81 examples/s]


In [85]:
paralel_377k = load_from_disk("dataset/paralel_3_lang/paralel_dataset_377k_dedup")

In [86]:
paralel_377k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 348695
})

In [87]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

def num_tokens_from_string(string: str, encoder) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = len(encoder.encode(string))
    return num_tokens

def count_tokens_in_dataset(dataset, field_name, num_tokens_from_string, encoder):
    total_tokens = 0
    
    for item in dataset:
        text = item[field_name]
        tokens = num_tokens_from_string(text, encoder)
        total_tokens += tokens
    
    return total_tokens

In [128]:
paralel_400k = load_from_disk("dataset/paralel_3_lang/paralel_dataset_400k_dedup")

In [129]:
print(count_tokens_in_dataset(paralel_400k, "indonesian", num_tokens_from_string, encoding))

168216484


In [130]:
print(count_tokens_in_dataset(paralel_400k, "cirebonese", num_tokens_from_string, encoding))

182488277


In [131]:
print(count_tokens_in_dataset(paralel_400k, "balinese", num_tokens_from_string, encoding))

184671868


In [249]:
paralel_400k = load_from_disk("dataset/id_hq/id_hq_data_prompt_300k")

In [266]:
import random

random.choice(paralel_400k)

{'text': 'Joel Robles Blázquez (lahir 17 Juni 1990), sing dikenal kanthi jeneng Joel, ya iku pemain bal-balan profésional Spanyol sing main dadi kiper kanggo klub Liga Utama Inggris Everton F.C. Robles miwiti karir profésional ing Atlético Madrid, wiwitané ing tim C ing taun 2008, sadurungé maju ing tim B ing taun sabanjuré, banjur diunggahaké dadi pemain senior. Dhèwèké dipindhah menyang Rayo Vallecano ing Januari 2012, lan setaun sangarepé nggabung karo klub Liga Utama Wigan Athletic, ing endi dhèwèké dadi bagéyan saka pamenang FA Cup Final 2013 sadurungé gabung karo Everton.\n\nRobles main ing pertandhingan internasional kanggo Spanyol ing tingkat U-16, U-17, U-21 lan U-23, lan dadi bagéan saka skuad sing menang ing Kejuaraan Eropah U-21 2013.\n\nKarir klub\n\nAtlético Madrid\nLair ing Getafe, Madrid, Robles kalebu asil produk pelatihan remaja ing Kutha Getafe. Dhèwèké gabung karo Atlético Madrid ing taun 2005. Dhèwèké ngrampungaké mangsa profésional kapisané ing mangsa (2009-10), d

In [248]:
random.choice(paralel_300k)

{'id': 'a1c3096e-943c-4787-9870-18017e6a701b',
 'cirebonese': 'Ari 2002 XG4 mangrupa hiji astéroid. Ieu asteroid téh bagéan tina astéroid Amor, anu nganjrek deukeut jeung marcapada. Ékséntrisitas orbit ieu astéroid kacatet gedéna 0.480, sedengkeun magnitudo mutlakna 18.2. Ari nu cios référénsina mah nyaéta MPO 237836.\n\nBebentukan\nKawas sakumna astéroid, ieu astéroid kabentuk tina nébula panonpoé primordial minangka beubeulahan planétisimal, objék di nébula marcapada ngora nu teu cukup badag pikeun robah jadi planét.\n\nRujukan\n\nTutumbu kaluar\nDaptar astéroid Amor - The sejagat Astronomical Union Minor Planet Center.\n\n338347\n338347',
 'indonesian': 'Ari 2002 XG4 mangrupa hiji astéroid. Ieu asteroid téh bagéan tina astéroid Amor, anu nganjrek deukeut jeung marcapada. Ékséntrisitas orbit ieu astéroid kacatet gedéna 0.480, sedengkeun magnitudo mutlakna 18.2. Ari nu jadi référénsina mah nyaéta MPO\xa0237836.\n\nBebentukan\nKawas sakumna astéroid, ieu astéroid kabentuk tina nébula p

In [1]:
import fasttext

# load the model
model = fasttext.load_model("./models/glotlid/model.bin")

In [3]:
import fasttext
import numpy as np
class CustomLID:
    def __init__(self, model_path, languages = -1, mode='before'):
        self.model = fasttext.load_model(model_path)
        self.output_matrix = self.model.get_output_matrix()
        self.labels = self.model.get_labels()
        
        # compute language_indices
        if languages !=-1 and isinstance(languages, list):
            self.language_indices = [self.labels.index(l) for l in list(set(languages)) if l in self.labels]

        else:
            self.language_indices = list(range(len(self.labels)))

        # limit labels to language_indices
        self.labels = list(np.array(self.labels)[self.language_indices])
        
        # predict
        self.predict = self.predict_limit_after_softmax if mode=='after' else self.predict_limit_before_softmax

    
    def predict_limit_before_softmax(self, text, k=1):
        
        # sentence vector
        sentence_vector = self.model.get_sentence_vector(text)
        
        # dot
        result_vector = np.dot(self.output_matrix[self.language_indices, :], sentence_vector)

        # softmax
        softmax_result = np.exp(result_vector - np.max(result_vector)) / np.sum(np.exp(result_vector - np.max(result_vector)))

        # top k predictions
        top_k_indices = np.argsort(softmax_result)[-k:][::-1]
        top_k_labels = [self.labels[i] for i in top_k_indices]
        top_k_probs = softmax_result[top_k_indices]

        return tuple(top_k_labels), top_k_probs


    def predict_limit_after_softmax(self, text, k=1):
        
        # sentence vector
        sentence_vector = self.model.get_sentence_vector(text)
        
        # dot
        result_vector = np.dot(self.output_matrix, sentence_vector)

        # softmax
        softmax_result = np.exp(result_vector - np.max(result_vector)) / np.sum(np.exp(result_vector - np.max(result_vector)))

        # limit softmax to language_indices
        softmax_result = softmax_result[self.language_indices]

        
        # top k predictions
        top_k_indices = np.argsort(softmax_result)[-k:][::-1]
        top_k_labels = [self.labels[i] for i in top_k_indices]
        top_k_probs = softmax_result[top_k_indices]

        return tuple(top_k_labels), top_k_probs


In [8]:
# to make sure these languages are available in GlotLID check the list of supported labels in model.labels
limited_languages = ['__label__ind_Latn', '__label__sun_Latn', '__label__jav_Latn', '__label__ban_Latn']

model = CustomLID("./models/glotlid/model.bin", languages = limited_languages , mode='before')

In [75]:
pred = model.predict("halo halo bandung ibu kota priangan", 3)

In [246]:
import random

model.predict(random.choice(paralel_705k)["balinese"].split("\n")[-1])

(('__label__ind_Latn',), array([0.99998975], dtype=float32))

In [55]:
import datasets

# 1. Load the dataset
paralel_705k = datasets.load_from_disk("dataset/paralel_3_lang/combined_paralel_dataset_705k_dedup")

In [247]:
def filter_non_id_text(data):
    sentences = data["indonesian"].split("\n")
    first_line = sentences[0]
    first_pred = model.predict(first_line)[0][0]
    if len(sentences) == 1:
      if first_pred == "__label__ind_Latn":
          return None
      else:
          return data
    else:
      second_line = sentences[1]
      second_pred = model.predict(second_line)[0][0]
      if first_pred == "__label__ind_Latn" and second_pred == "__label__ind_Latn":
          return None
      else:
          return data

def filter_id_text(data):
    sentences = data["indonesian"].split("\n")
    first_line = sentences[0]
    first_pred = model.predict(first_line)[0][0]
    if len(sentences) == 1:
      if first_pred == "__label__ind_Latn":
          return data
      else:
          return None
    else:
      second_line = sentences[1]
      second_pred = model.predict(second_line)[0][0]
      if first_pred == "__label__ind_Latn" and second_pred == "__label__ind_Latn":
          return data
      else:
          return None

def filter_id_text_2(data):
    sentences = data["text"].split("\n")
    first_line = sentences[0]
    first_pred = model.predict(first_line)[0][0]
    if len(sentences) == 1:
      if first_pred == "__label__ind_Latn":
          return data
      else:
          return None
    else:
      second_line = sentences[1]
      second_pred = model.predict(second_line)[0][0]
      if first_pred == "__label__ind_Latn" and second_pred == "__label__ind_Latn":
          return data
      else:
          return None

def filter_all_text(data):
    cbn_sentences = data["cirebonese"].split("\n")
    ban_sentences = data["balinese"].split("\n")
    
    cbn_first_line = cbn_sentences[0]
    cbn_first_pred = model.predict(cbn_first_line)[0][0]
    cbn_verdict = False
    if len(cbn_sentences) == 1:
      if cbn_first_pred == "__label__jav_Latn":
          cbn_verdict = True
    else:
      cbn_second_line = cbn_sentences[1]
      cbn_second_pred = model.predict(cbn_second_line)[0][0]
      if cbn_first_pred == "__label__jav_Latn" or cbn_second_pred == "__label__jav_Latn":
          cbn_verdict = True

    ban_first_line = ban_sentences[0]
    ban_first_pred = model.predict(ban_first_line)[0][0]
    ban_verdict = False
    if len(ban_sentences) == 1:
      if ban_first_pred == "__label__ban_Latn":
          ban_verdict = True
    else:
      ban_second_line = ban_sentences[1]
      ban_second_pred = model.predict(ban_second_line)[0][0]
      if ban_first_pred == "__label__ban_Latn" or ban_second_pred == "__label__ban_Latn":
          ban_verdict = True

    if cbn_verdict and ban_verdict:
        return data
    else:
        return None

In [248]:
clean_paralel_705k = paralel_705k.filter(filter_all_text, num_proc=8)

Filter (num_proc=8): 100%|██████████| 557859/557859 [01:21<00:00, 6808.73 examples/s]


In [250]:
clean_paralel_705k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 491113
})

In [251]:
clean_paralel_705k.save_to_disk("dataset/paralel_3_lang/combined_paralel_dataset_705k_dedup_clean")

Saving the dataset (5/5 shards): 100%|██████████| 491113/491113 [00:19<00:00, 25814.94 examples/s]


In [46]:
paralel_305k = datasets.load_from_disk("dataset/id_hq/id_hq_data_prompt_305k")

In [52]:
clean_paralel_305k = paralel_305k.filter(filter_id_text_2, num_proc=8)

Filter (num_proc=8): 100%|██████████| 305224/305224 [00:48<00:00, 6238.72 examples/s]


In [54]:
clean_paralel_305k.save_to_disk("dataset/id_hq/id_hq_data_prompt_305k_clean")

Saving the dataset (0/8 shards):   0%|          | 0/264918 [00:00<?, ? examples/s]

Saving the dataset (8/8 shards): 100%|██████████| 264918/264918 [00:34<00:00, 7772.17 examples/s] 


In [253]:
def is_text_too_long(example):
    return len(example['indonesian'].split()) > 32000

In [294]:
txt = random.choice(clean_paralel_705k)['indonesian']
len(txt.split())

87